# Algorithme des k plus proches voisins
l’algorithme des k plus proches voisins (K-NN où K-nearest neighbors) est une méthode utilisée pour la classification de données. Son fonctionnement peut être assimilé à l’analogie suivante *dis moi qui sont tes voisins, je te dirais qui tu es…*.

Dans cette activité, vous allez travailler sur un jeu de données fictives : 30 points sont disposés sur un plateau de taille $100 \times 100$. Ces points sont de couleur *rouge*, *verte*, *bleue* ou *noire*. 

Le problème est le suivant : étant données les coordonnées d'un point choisi au hasard sur notre plateau, peut-on deviner sa couleur ? La réponse à cette question sera apportée par l'algorithme des k plus proches voisins que vous devrez réaliser en suivant les indications ci-dessous.

In [ ]:
# Import des bibliothèques nécessaires

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from random import randint
from math import sqrt
import hashlib
import sys
%matplotlib inline

In [ ]:
# Constantes
MAX_RANGE = 100                        # les coordonnées sont entre 0 et 99
PRECISION = 10*sys.float_info.epsilon  # precision des tests sur les flottants
FORMES = {"red":"o", "blue":"^", "green":"s", "black":"x"}
COULEURS = {'red':(255,0,0), 'green':(0,255,0), 'blue':(0,0,255), 'black':(0,0,0)}

## Jeu de données

les informations de départ sont données sous forme d'un dictionnaire où
   - la clé est la propriété étudiée, à savoir une couleur parmi "red", "green", "blue", "black"
   - la valeur est une liste contenant les coodonnées des points (x,y) dont la couleur est spécifiée dans la clé

### Récupération d'un jeu existant
Validez les 3 cellules ci-dessous et étudiez la manière dont les données sont représentées dans le dictionnaire ***data***

In [ ]:
def nb_data(data):
    """Renvoie le nombre de points total dans le jeu de données"""
    return sum([len(data[propriete]) for propriete in data.keys()])

In [ ]:
data = {
 'black': [(52, 2), (59, 0), (68, 8), (77, 11), (28, 21)],
 'blue': [(0, 66), (0, 77), (28, 61), (21, 56), (29, 66),(10, 40),(10, 81),(30, 49)],
 'green': [(83, 87), (96, 56), (45, 77), (73, 69), (53, 77), (34, 68), (70, 63)],
 'red': [(85, 34), (36, 54), (33, 54), (65, 52),  (45, 45), (70, 62), (48, 66), (63, 28), (40, 28),(82, 18)]
       }

In [ ]:
print("ce jeu contient ", nb_data(data), " données")
print("les points noirs sont ", data["black"])

### Représentation graphique du jeu de données

Pour visualiser notre jeu de données, chaque donnée sera un point dont les coordonnées seront les deux valeurs entre 0 et 100 et dont la couleur et la forme corrspondront au critère associé. Pour simplifier la lecture sur un document noir et blanc, nous adopterons la convention suivante :
- le rouge sera représenté par un point rond
- le vert sera représenté par un carré
- le bleu sera représenté par un triangle
- le noir sera représenté par une croix

In [ ]:
def dessine_points(data):
    plt.rcParams["figure.figsize"] = (4, 4)
    plt.axis('equal')
    for couleur in data.keys():
        x=[e[0] for e in data[couleur]]
        y=[e[1] for e in data[couleur]]
        plt.scatter(x, y, c=couleur, marker=FORMES[couleur])
    plt.show()
dessine_points(data)

## Implémentation de l'algorithme des k plus proches voisins

On cherche dans cette partie à écrire un programme qui, étant données les coordonnées d'un point, devine la couleur associée en utilisant l'algorithme des k plus proches voisins.

### Calcul de la distance entre deux points
Ecrire une fonction **distance** prenant en paramètre deux points (un point sera représenté par un tuple (x,y)) et renvoyant la distance entre ces deux points.

In [ ]:
### BEGIN SOLUTION
def distance(pointA, pointB):
    xA, yA = pointA
    xB, yB = pointB
    return sqrt((xB-xA)**2 + (yB-yA)**2)
### END SOLUTION

In [ ]:
# test de la fonction
origine = (5, 5)
voisin1 = (3, 4)
voisin2 = (0, 0)
print(distance(origine, voisin1))
print(distance(origine, voisin2))

# Verification automatisee
assert abs(distance(origine, voisin1) - 2.23606797749979) < PRECISION
assert abs(distance(origine, voisin2) - 7.0710678118654755) < PRECISION
### BEGIN HIDDEN TESTS
assert abs(distance((34,52), (12, 30)) - 31.11269837220809) < PRECISION
assert abs(distance((34,52), (34, 52)) - 0.0) < PRECISION
### END HIDDEN TESTS

### Calcul des distances de tous les points à une origine donnée

Construire une fonction ***k_proches_voisins*** qui prend en arguments
- leu jeu de données
- les coordonnées du point origine dont on cherche à déterminer a nature
- la valeur de *k* : nombre de plus proches voisins à considérer

et renvoyant la liste des k plus proches voisins sous forme d'un tuple (distance, propriete)

In [ ]:
### BEGIN SOLUTION
def k_proches_voisins(data, origine, k=1):
    reponse=[]
    for propriete in data.keys():
        for voisin in data[propriete]:
            d = distance(origine, voisin)
            # Insertion dans la liste des k plus proches voisins
            i=0
            while i< k and i < len(reponse) and reponse[i][0] <= d:
                i+=1
            if i < k:
                reponse.insert(i,(d, propriete))
    return reponse[:k]
### END SOLUTION

In [ ]:
# test de la fonction
reponse = k_proches_voisins(data, (5,5), 5)
for r in reponse:
    print(r)
# tests automatises
assert abs(reponse[0][0]-28.0178514522438) < PRECISION
assert reponse[0][1] == 'black'
assert abs(reponse[1][0]-35.35533905932738) < PRECISION
assert reponse[1][1] == 'blue'
assert abs(reponse[2][0]-41.88078318274385) < PRECISION
assert reponse[2][1] == 'red'
assert abs(reponse[3][0]-47.095647357266465) < PRECISION
assert reponse[3][1] == 'black'
assert abs(reponse[4][0]-50.60632371551998) < PRECISION
assert reponse[4][1] == 'blue'
### BEGIN HIDDEN TESTS
reponse1 = k_proches_voisins(data, (51,25), 2)
assert abs(reponse1[0][0]-11.40175425099138) < PRECISION
assert reponse1[0][1] == 'red'
assert abs(reponse1[1][0]-12.36931687685298) < PRECISION
assert reponse1[1][1] == 'red'
### END HIDDEN TESTS

### Prédiction de la valeur du caractère en fonction des k plus proches voisins

Déduire de la fonction précédante la fonction ***categorie_devine*** qui prend en arguments
- le jeu de données
- les coordonnées du point origine dont on cherche à déterminer a nature
- la valeur de *k* : nombre de plus proches voisins à considérer
et qui renvoie la propriété estimée par l'algorithme des k plus proches voisins, à savoir la propriété la plus fréquente parmis les $k$ plus proches voisins. 

En cas d'égalité, on choisira la couleur dont la **somme des distances correspondante est la plus petite**.

**Indication** : On pourra procéder en plusieurs étapes et créer des fonctions réalisant des tâches plus simple comme par exemple chercher pour chaque couleur le nombre d'apparition dans la liste des k plus proches voisins.

In [ ]:
### BEGIN SOLUTION
def categorie_devine(data, origine, k=1):
    liste_cles = list(data.keys())
    tri = [[0,0,p] for p in liste_cles]
    
    kpv = k_proches_voisins(data, origine, k)
    
    for distance, propriete in kpv:
        i = liste_cles.index(propriete)
        tri[i][0] -= 1
        tri[i][1] += distance
    return min(tri)[2]
### END SOLUTION

In [ ]:
# test de la fonction
reponse = categorie_devine(data, (5,5),5)
print(reponse)
# test automatise
assert categorie_devine(data, ( 51 , 42 ),6) == 'red'
assert categorie_devine(data, ( 40 , 40 ),3) == 'red'
assert categorie_devine(data, ( 40 , 60 ),3) == 'red'
assert categorie_devine(data, ( 40 , 80 ),3) == 'green'
assert categorie_devine(data, ( 60 , 20 ),3) == 'black'
assert categorie_devine(data, ( 60 , 40 ),30) == 'red'
### BEGIN HIDDEN TESTS
assert categorie_devine(data, ( 20 , 20 ),3) == 'black'
assert categorie_devine(data, ( 20 , 40 ),3) == 'blue'
assert categorie_devine(data, ( 20 , 60 ),3) == 'blue'
assert categorie_devine(data, ( 20 , 80 ),3) == 'blue'
assert categorie_devine(data, ( 40 , 20 ),3) == 'black'
assert categorie_devine(data, ( 60 , 60 ),3) == 'red'
assert categorie_devine(data, ( 60 , 80 ),3) == 'green'
assert categorie_devine(data, ( 80 , 20 ),3) == 'red'
assert categorie_devine(data, ( 80 , 40 ),3) == 'red'
assert categorie_devine(data, ( 80 , 60 ),3) == 'green'
assert categorie_devine(data, ( 80 , 80 ),3) == 'green'
### END HIDDEN TESTS

## Régionnement du plan

En partant de la représentation graphique de notre jeu de données, nous allons colorier chaque point du plan avec la couleur prédite par l'algorithme des k plus proches voisins. Nous visualiserons l'influence du paramètre $k$ dans la prédiction obtenue.

L'idée dans cette partie est la suivante : on parcourt tous les points du plan et on colorie le pixel avec la couleur prédite par l'algorithme en fonction de ses k plus proches voisins.

Pour cette tâche, nous aurons besoin de savoir comment créer une image pixels par pixels. Etudiez l'exemple ci-dessous qui vous montrera l'utilisation de la librairie python *PIL* dédiée à cela.

In [ ]:
# On cree une image vide
monimage=Image.new("RGB",(MAX_RANGE, MAX_RANGE))

# On definit une couleur selon les valeurs Rouge, Vert et Bleu entre 0 et 255
R,V,B = (230,64,21)

# On calcule une image de 100 pixels par 100 pixels
for x in range(MAX_RANGE):
    for y in range(MAX_RANGE):
        monimage.putpixel((x,MAX_RANGE-y-1),(R,V,B))

# On zoome sur l'image pour lui donner la taille demandée
monimage.resize((240,240), Image.ANTIALIAS)

L'image ci-dessus vous semble trop monotonne ? amusez-vous à modifier la couleur des pixels en fonction des coordonnées. Par exemple : mettez $2x$ pour le canal vert et $2y$ pour le canal bleu.

**Remarque** : On rappelle que l'origine du repère dans *PIL* est le point en haut à gauche et l'axe des ordonnées est orienté vers le bas.
Dans *matplotlib*, l'origine du repère est le point en bas à gauche et l'axe des ordonnées est orienté vers le haut. 
On tiendra compte de ces spécificités de manière à ce que le régionnement obtenu corresponde à la représentation des points obtenue ci-dessus.


### Régionnement du plan

Ecrire une fonction ***remplissage*** qui prend en paramètre le jeu de données et la valeur de $k$ et qui renvoie une image du plan colorié avec une résolution de 240x240 pixels.

**Indication** : On pourra utiliser le dictionnaire **COULEURS** donné en constante au début de ce document pur convertir les propriétés de couleur en valeurs au format (R,V,B).

**Attention** : Penser à renvoyer l'image fabriquée (avec *return*) avec les dimensions demandées à la fin de la fonction ***remplissage*** !

In [ ]:
### BEGIN SOLUTION
def remplissage(data, k):
    monimage=Image.new("RGB",(MAX_RANGE, MAX_RANGE))
    for x in range(MAX_RANGE):
        for y in range(MAX_RANGE):
            couleur=categorie_devine(data, (x,y), k)
            monimage.putpixel((x,MAX_RANGE-y-1),COULEURS[couleur])
    return monimage.resize((240,240), Image.ANTIALIAS)
### END SOLUTION

In [ ]:
# test automatise
monImage = remplissage(data, 3)
assert monImage.size == (240,240)
assert type(monImage) == Image.Image
### BEGIN HIDDEN TESTS
assert hashlib.md5(monImage.tobytes()).hexdigest() == 'c026d28f9865baa13d6df74c9fc91ccb'
### END HIDDEN TESTS
monImage

Exécutez cette fonction avec plusieurs valeurs de $k$ variant entre 4 et 10

In [ ]:
remplissage(data, 4)

In [ ]:
remplissage(data, 5)

In [ ]:
remplissage(data, 6)

In [ ]:
remplissage(data, 7)

In [ ]:
remplissage(data, 8)

In [ ]:
remplissage(data, 9)

In [ ]:
remplissage(data, 10)

# Pour aller plus loin

Il est intéressant de constater l'influence du paramètre $k$ sur les figures obtenues. Lorsque k évolue, les frontières bougent, des ilôts se forment et de défont.

## Choix du paramètre k

Le choix du bon paramètre $k$ est un problème délicat : 
- Une trop faible valeur fera apparaître beaucoup de distortions liées au hasard et donc des choix erronés au final.
- Une trop grande valeur de k prendra en compte des valeur très éloignées du sujet d'origine et donnera encore des résultats peu crédibles.

**Question** : Que renverra l'algorithme des $k$ plus proches voisins lorsque on choisit le paramètre $k$ égal au nombre de points dans notre jeu de données ?

Afin d'avoir des résultats pertinents par note algorithme, le choix du paramètre $k$ devra se faire entre ces deux extrèmes. Il n'y a pas de règle absolue, néanmoins on peut retenir deux idées :
1. Si le nombre de critères est égal à 2, on choisira ***une valeur de k impaire*** afin d'éviter les situations d'égalités.
2. En première approximation, on pourra choisir $k=\sqrt n$ où $n$ est le nombre de points dans notre jeu de données.

Dans l'exemple ci-dessus, le paramètre $k=6$ semble donner des résultats assez proches de ce que notre intuition suggère, avec des frontières assez lisses. Lorsque $k$ augmente, le rouge prend une prédominance exagérée.

## Influence du paramètre k
On a constaté que, lorsque k varie,  les frontières bougent mais beaucoup de points restent stables. On peut les mettre en évidence avec un système de dégradé de couleurs. L'image que l'on cherche à obtenir est en quelque sorte une superposition des images obtenues précédemment.

In [ ]:
def invariants(data, start, end):
    liste = [[[0,0,0,0] for _ in range(MAX_RANGE)] for _ in range(MAX_RANGE) ]
    couleurs_position = dict({'red':0, 'green':1, 'blue':2, 'black':3})
    for k in range(start, end):
         for x in range(MAX_RANGE):
            for y in range(MAX_RANGE):
                c=categorie_devine(data, (x,y), k)
                liste[y][x][couleurs_position[c]] += 1
             
    return liste

def calcule_couleur(liste_col, nb_max):
    return tuple(c *255 // nb_max // 2**liste_col[3] for c in liste_col[:3])

def dessin_invariants(liste):
    nb_max=sum(liste[0][0])
    monimage=Image.new("RGB",(MAX_RANGE, MAX_RANGE))
    for x in range(MAX_RANGE):
        for y in range(MAX_RANGE):
            couleur = calcule_couleur(liste[y][x], nb_max)
            monimage.putpixel((x,MAX_RANGE-y-1),couleur)
    return monimage.resize((240,240), Image.ANTIALIAS)

In [ ]:
liste = invariants(data, 3, 8)

In [ ]:
dessin_invariants(liste)

## création aléatoire du jeu de données
Pour fabriquer un nouveau jeu de données, on pourra utiliser la cellule suivante. On utilise une loi de probabilité normale *en forme de cloche* afin d'obtenir des données assez regroupées en zones :
- les paramètres mu1 et mu2 représentent le sommet de la cloche concentrant le maximum de probabilité (espérance)
- les paramètres sigma1 et sigma2 représentent la dispertion des données (écart-type)

In [ ]:
data1=dict() # nouveau jeu de données

def aleat(mu, sigma):
    return min(MAX_RANGE,max(0,int(np.random.normal(mu, sigma))))

def creeData(key, nb, mu1, sigma1, mu2, sigma2):
    data1[key]=[(aleat(mu1, sigma1), aleat(mu2, sigma2)) for _ in range(nb) ]

creeData("red", 10, 50, 20, 50, 20)  # points rouges concentrées autour de (50,50)
creeData("blue", 8, 10, 20, 70, 10)  # points bleus concentrées autour de (20,20)
creeData("green", 5, 80, 20, 90, 10) # points verts concentrées autour de (70,70)
creeData("black", 7, 70, 20, 20, 20) # points noirs concentrées autour de (70,20)
dessine_points(data1)

In [ ]:
remplissage(data1, 6)